In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (
    Dense,
    Dropout,
    BatchNormalization,
    Input,
    LeakyReLU,
    Flatten,
)
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# 加载MNIST数据集
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 数据预处理
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

x_train = np.expand_dims(x_train, axis=-1)  # 增加通道维度
x_test = np.expand_dims(x_test, axis=-1)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 数据增强
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    zoom_range=0.5,
    shear_range=0.5,
    fill_mode="nearest",
)
datagen.fit(x_train)

# 构建多层感知机（MLP）神经网络模型
model = Sequential(
    [
        Flatten(input_shape=(28, 28, 1)),

        Dense(1024), # Dense 全连接层
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        
        Dense(1024),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        
        # ------- 做两层完全一样的 强化提取特征值 -------
        
        Dense(512),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(256),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(128),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(64),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(32),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        # ------------------ 输出 ------------------
        Dense(10, activation="softmax"),  # 输出层
    ]
)

# 编译模型
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),  # 使用RMSprop优化器
    loss="categorical_crossentropy",  # 多分类交叉熵损失
    metrics=["accuracy"],
)

# 回调函数
# 在损失函数平稳时减小学习率
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=8, min_lr=1e-6, verbose=1
)

# 提前停止 达到最优值时停止训练
early_stopping = EarlyStopping(
    monitor="val_loss", patience=15, restore_best_weights=True, verbose=1
)

# 训练模型
model.fit(
    datagen.flow(x_train, y_train, batch_size=64),
    epochs=300,
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // 64,
    callbacks=[reduce_lr, early_stopping],
)

# 评估模型
test_loss, test_acc = model.evaluate(x_test, y_test)  # 使用调整后的测试数据形状
print(f"Test accuracy: {test_acc}")

# 保存模型
model.save("mnist_model_mlp_v1.keras")

2025-01-12 22:43:26.708490: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-01-12 22:43:26.708522: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-01-12 22:43:26.708529: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-01-12 22:43:26.708561: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-12 22:43:26.708578: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/300


2025-01-12 22:43:27.835757: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


935/937 [============================>.] - ETA: 0s - loss: 2.2863 - accuracy: 0.1894

2025-01-12 22:43:53.060161: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


937/937 [==============================] - 28s 26ms/step - loss: 2.2857 - accuracy: 0.1895 - val_loss: 1.7299 - val_accuracy: 0.3913 - lr: 0.0010
Epoch 2/300
937/937 [==============================] - 24s 26ms/step - loss: 1.8126 - accuracy: 0.3418 - val_loss: 1.1754 - val_accuracy: 0.5407 - lr: 0.0010
Epoch 3/300
937/937 [==============================] - 24s 25ms/step - loss: 1.5492 - accuracy: 0.4576 - val_loss: 0.9117 - val_accuracy: 0.6707 - lr: 0.0010
Epoch 4/300
937/937 [==============================] - 24s 25ms/step - loss: 1.3950 - accuracy: 0.5185 - val_loss: 0.7789 - val_accuracy: 0.7358 - lr: 0.0010
Epoch 5/300
937/937 [==============================] - 24s 25ms/step - loss: 1.2880 - accuracy: 0.5638 - val_loss: 0.6735 - val_accuracy: 0.8001 - lr: 0.0010
Epoch 6/300
937/937 [==============================] - 24s 26ms/step - loss: 1.1851 - accuracy: 0.6129 - val_loss: 0.4725 - val_accuracy: 0.8657 - lr: 0.0010
Epoch 7/300
937/937 [==============================] - 24s 26ms/